# 🍬 [Day 10](https://adventofcode.com/2019/day/10)

In [1]:
import numpy as np
from collections import defaultdict

def parse_asteroids_field(inputs):
    """Parse the inputs"""
    asteroids = []
    for i, line in enumerate(inputs):
        for j, c in enumerate(line):
            if c == '#':
                asteroids.append((i, j))
    return np.array(asteroids, dtype=np.float32) 


def find_best_locations(asteroids):
    """Find best station location"""
    num_views = 0
    station = (0, 0)
    flt = np.ones((asteroids.shape[0],))
    for i, (x, y) in enumerate(asteroids):
        # Switch to polar coordinates
        polar_angle = np.arctan2(asteroids[:, 0] - x, asteroids[:, 1] - y)
        # Count unique asteroids, ignoring self
        s = len(np.unique(polar_angle))
        # Update
        if s > num_views:
            num_views = s
            station = (x, y)
    return num_views, station


def vaporize(asteroids, station, num=200):
    """Vaporize asteroids until reaching reqired count"""
    # Polar coordinates
    # invert y and x to sort from up direction and clockwise
    x0, y0 = station
    polar_angle = np.arctan2(asteroids[:, 1] - y0, asteroids[:, 0] - x0)
    radius = np.sqrt((asteroids[:, 0] - x0)**2 + (asteroids[:, 1] - y0)**2)
    
    # Store asteroids per angle and sorted per radius
    ordered = defaultdict(lambda: [])
    for a, r in zip(polar_angle, radius):
        if r > 0.:  # ignore the station itself
            ordered[a].append(r)
    for a in ordered:
        ordered[a] = sorted(ordered[a])
        
    # Start to vaporize
    vaporized = 0
    while vaporized < num:
        rotation = sorted(ordered, key=lambda x: -x) 
        for a in rotation:
            vaporized += 1
            if vaporized == num:
                y = np.round(ordered[a][0] * np.cos(a) + x0)
                x = np.round(ordered[a][0] * np.sin(a) + y0)
                return int(x), int(y)
            ordered[a].pop()
            if len(ordered[a]) == 0:
                del ordered[a]

In [2]:
with open("inputs/day10.txt", 'r') as f:
    inputs = f.read().splitlines()
    asteroids = parse_asteroids_field(inputs)

    
views, station = find_best_locations(asteroids)
print("The best locations sees {} asteroids".format(views))

print("The 200th vaporized asteroid has coordinates {}".format(vaporize(asteroids, station, num=200)))

The best locations sees 230 asteroids
The 200th vaporized asteroid has coordinates (12, 5)
